In [1]:
# pixi run py-build-examples
# pixi run -e examples py-build-notebook
# pixi run -e examples jupyter notebook /path/to/dataplatform/Indexing.ipynb

In [1]:
from __future__ import annotations

import os

# os.environ["RERUN_NOTEBOOK_ASSET"] = "serve-local"
os.environ["RERUN_NOTEBOOK_ASSET"] = "inline"

In [2]:
import rerun as rr

import pyarrow as pa
import pandas as pd
import numpy as np
import datafusion

from rerun_bindings import ViewerClient

DEV ENVIRONMENT DETECTED! Re-importing rerun from: /Users/goertler/Developer/rerun-io/rerun/rerun_py/rerun_sdk


### Connect to the Rerun viewer using new API

In [8]:
# client = ViewerClient(addr="rerun+http://0.0.0.0:9876")

In [3]:
viewer = rr.notebook.Viewer(width=1024, height=700)
viewer.display()

Viewer()

### Send a basic table

In [4]:
viewer.send_table(
    "Hello from Notebook",
    pa.RecordBatch.from_pydict({"Column A": [1, 2, 3], "Column B": ["https://www.rerun.io", "Hello", "World"]}),
)

### Send a Pandas Dataframe

In [34]:
dates = pd.date_range("20130101", periods=6)
df = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list("ABCD"))
df_reset = df.reset_index().rename(columns={'index': 'date'})
df

,A,B,C,D
2013-01-01,-0.167462,-1.908861,-2.558886,0.872520
2013-01-02,-1.319389,0.774815,-0.393330,-0.895299
2013-01-03,-1.292820,-0.431619,-0.740074,1.202464
2013-01-04,-0.293862,-0.936821,-0.135840,0.341631
2013-01-05,1.743247,2.041301,1.658495,-0.820803
2013-01-06,-0.069138,0.309457,-0.611314,-0.810606


In [35]:
client.send_table("Hello from Pandas", pa.RecordBatch.from_pandas(df))

### Can we do DataFusion?

In [19]:
datafusion.__version__

'45.2.0'

In [20]:
from datafusion import SessionContext

In [21]:
ctx = SessionContext()

In [36]:
df = ctx.read_csv("pokemon.csv")
df.show()

DataFrame()
+----+---------------------------+--------+--------+-------+----+--------+---------+---------+---------+-------+------------+-----------+
| #  | Name                      | Type 1 | Type 2 | Total | HP | Attack | Defense | Sp. Atk | Sp. Def | Speed | Generation | Legendary |
+----+---------------------------+--------+--------+-------+----+--------+---------+---------+---------+-------+------------+-----------+
| 1  | Bulbasaur                 | Grass  | Poison | 318   | 45 | 49     | 49      | 65      | 65      | 45    | 1          | false     |
| 2  | Ivysaur                   | Grass  | Poison | 405   | 60 | 62     | 63      | 80      | 80      | 60    | 1          | false     |
| 3  | Venusaur                  | Grass  | Poison | 525   | 80 | 82     | 83      | 100     | 100     | 80    | 1          | false     |
| 3  | VenusaurMega Venusaur     | Grass  | Poison | 625   | 80 | 100    | 123     | 122     | 120     | 80    | 1          | false     |
| 4  | Charmander     

In [40]:
# Limitation: We currently only support a single batch — we probably want to allow streaming multiple batches in the future.
client.send_table("Hello from Datafusion", df.collect()[0])

### Next?

* Finish everything
  * show_notebook
  * still some bugs
  * UI problems around closing recordings
 


In [ ]:
### Stretch
* Make `TableStore` a Datafusion provider
* Show summary statistics ala DuckDb UI